In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from talib import BBANDS, RSI, MA_Type, ATR, MFI
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# # Download historical data
# # Define the start and end dates for the data
# end_date = dt.datetime.now()
# start_date = end_date - dt.timedelta(days=729)

# # Download 5-minute interval data for the specified date range
# df = yf.download('TSLA', start=start_date, end=end_date, interval='1h')

# print(df.head())

tickerSymbol = 'msft'
# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)
# Get the historical prices for this ticker
df = tickerData.history(period='60d', interval='5m')
# Print the data
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-03-14 09:30:00-04:00,256.750000,257.940002,256.644989,257.119995,1929215,0.0,0.0
2023-03-14 09:35:00-04:00,257.170013,257.579987,255.860001,257.049988,728645,0.0,0.0
2023-03-14 09:40:00-04:00,257.100006,258.489990,257.086609,257.840088,449404,0.0,0.0
2023-03-14 09:45:00-04:00,257.869995,258.299011,257.279999,257.410004,342208,0.0,0.0
2023-03-14 09:50:00-04:00,257.359985,257.720001,257.179993,257.290009,263527,0.0,0.0


In [3]:
# Calculate Bollinger Bands
df['upper_band'], df['middle_band'], df['lower_band'] = BBANDS(df['Close'], timeperiod=14)

# Calculate RSI
df['rsi'] = RSI(df['Close'], timeperiod=14)

# Calculate Keltner Channels
df['keltner_middle'] = df['Close'].rolling(window=20).mean()
df['atr'] = ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
df['keltner_upper'] = df['keltner_middle'] + (df['atr'] * 1.5)
df['keltner_lower'] = df['keltner_middle'] - (df['atr'] * 1.5)

#Ca mfi
df['mfi'] = MFI(df['High'], df['Low'], df['Close'], df['Volume'])

In [4]:
# Set starting balance and risk amount
starting_balance = 10000
risk_amount = (.40 * starting_balance) # 5% of starting balance
stop_loss_pct = 0.15 # 10%
balance = starting_balance


In [5]:
# Initialize variables
position = 0
entry_price = 0
num_shares = 0

In [6]:
import pandas as pd

# Initialize trade journal
trade_journal = pd.DataFrame(columns=["Date", "Action", "Price", "Shares", "Balance"])


In [7]:
# Iterate over the data
for i, row in df.iterrows():
    # Check if RSI is below 20 and close price is below the lower Keltner Channel or lower Bollinger Band
    if row['rsi'] < 30 and (row['Close'] <  row['keltner_lower'] or row['Close'] < row['lower_band']):
        if position == 0:
            # Calculate the number of shares we can buy based on risk amount and current price
            num_shares = int(risk_amount / row['Close'])
            # Subtract the cost from our balance
            balance -= num_shares * row['Close']
            # Update our position
            position += num_shares
            # Update the entry price
            entry_price = row['Close']
            trade_journal = trade_journal.append({"Date": row.name, "Action": "Buy", "Price": row["Close"], "Shares": num_shares, "Balance": balance}, ignore_index=True)
 # Adjust sell condition
    elif ((row['rsi'] > 72 and (row['Close'] > row['keltner_upper'] or row['Close'] > row['upper_band'])) or
          (entry_price > 0 and row['Close'] < entry_price * (1 - stop_loss_pct)) or
          (row['mfi'] < 57 and row['rsi'] > 72)):  # added condition for MFI < 57 and RSI > 70
        if position > 0:
            # Add the revenue from selling our position to our balance
            balance += position * row['Close']
            trade_journal = trade_journal.append({"Date": row.name, "Action": "Sell", "Price": row["Close"], "Shares": position, "Balance": balance}, ignore_index=True)
            # Update our position
            position = 0

In [8]:
print(f'Final balance: {balance}')

Final balance: 6494.002410888672


In [9]:
# Separate dataframes for buy and sell operations
buy_df = trade_journal[trade_journal["Action"] == "Buy"].copy().reset_index(drop=True)
sell_df = trade_journal[trade_journal["Action"] == "Sell"].copy().reset_index(drop=True)

In [10]:
# Ensure that buy_df and sell_df have the same length
if len(buy_df) != len(sell_df):
    min_len = min(len(buy_df), len(sell_df))
    buy_df = buy_df[:min_len]
    sell_df = sell_df[:min_len]

In [11]:
# Calculate metrics
num_wins = (sell_df["Price"] > buy_df["Price"]).sum()
num_losses = (sell_df["Price"] < buy_df["Price"]).sum()
win_ratio = num_wins / (num_wins + num_losses)
total_profit = balance - starting_balance
num_trades = len(buy_df)

print(f'Number of trades: {num_trades}')
print(f'Win/Loss ratio: {win_ratio}')
print(f'Total profit/loss: {total_profit}')
print(trade_journal)

Number of trades: 14
Win/Loss ratio: 0.9285714285714286
Total profit/loss: -3505.997589111328
                        Date Action       Price Shares       Balance
0  2023-03-20 09:50:00-04:00    Buy  272.291901     14   6187.913391
1  2023-03-21 09:30:00-04:00   Sell  274.660004     14  10033.153442
2  2023-03-22 15:55:00-04:00    Buy  272.269989     14   6221.373596
3  2023-03-28 15:20:00-04:00   Sell  274.480011     14  10064.093750
4  2023-04-03 11:40:00-04:00    Buy  284.820007     14   6076.613647
5  2023-04-03 15:05:00-04:00   Sell  286.540009     14  10088.173767
6  2023-04-04 14:55:00-04:00    Buy  286.190002     13   6367.703735
7  2023-04-06 11:45:00-04:00   Sell  286.260010     13  10089.083862
8  2023-04-10 09:35:00-04:00    Buy  287.299988     13   6354.184021
9  2023-04-12 13:15:00-04:00   Sell  286.540009     13  10079.204132
10 2023-04-14 12:45:00-04:00    Buy  284.183014     14   6100.641937
11 2023-04-17 09:30:00-04:00   Sell  290.339996     14  10165.401886
12 2023-0